In [1]:
# !pip install optuna
# !pip install xgboost

In [2]:
import optuna
import xgboost as xg

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as iplot
import cufflinks as cf
cf.go_offline()

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [3]:
### loading the Admission Prediction dataset

df = pd.read_csv("https://raw.githubusercontent.com/divyansha1115/Graduate-Admission-Prediction/master/Admission_Predict.csv")
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82
396,397,325,107,3,3.0,3.5,9.11,1,0.84
397,398,330,116,4,5.0,4.5,9.45,1,0.91
398,399,312,103,3,3.5,4.0,8.78,0,0.67


In [4]:
df.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [5]:
df.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,200.500000,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,115.614301,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,100.750000,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,200.500000,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,300.250000,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,400.000000,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


In [6]:
### Null val check

df.isna().sum()

Serial No.           0
GRE Score            0
TOEFL Score          0
University Rating    0
SOP                  0
LOR                  0
CGPA                 0
Research             0
Chance of Admit      0
dtype: int64

In [7]:
# features

X = df.drop(columns=["Serial No.", "Chance of Admit "])
X

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337,118,4,4.5,4.5,9.65,1
1,324,107,4,4.0,4.5,8.87,1
2,316,104,3,3.0,3.5,8.00,1
3,322,110,3,3.5,2.5,8.67,1
4,314,103,2,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
395,324,110,3,3.5,3.5,9.04,1
396,325,107,3,3.0,3.5,9.11,1
397,330,116,4,5.0,4.5,9.45,1
398,312,103,3,3.5,4.0,8.78,0


In [8]:
# Label

Y = df["Chance of Admit "]

In [9]:
### Standard Scaling the data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled

array([[ 1.76210664,  1.74697064,  0.79882862, ...,  1.16732114,
         1.76481828,  0.90911166],
       [ 0.62765641, -0.06763531,  0.79882862, ...,  1.16732114,
         0.45515126,  0.90911166],
       [-0.07046681, -0.56252785, -0.07660001, ...,  0.05293342,
        -1.00563118,  0.90911166],
       ...,
       [ 1.15124883,  1.41704229,  0.79882862, ...,  1.16732114,
         1.42900622,  0.90911166],
       [-0.41952842, -0.72749202, -0.07660001, ...,  0.61012728,
         0.30403584, -1.09997489],
       [ 1.41304503,  1.58200646,  0.79882862, ...,  0.61012728,
         1.78160888,  0.90911166]])

In [10]:
## For cross validation

def objective(trial, data=X_scaled, target=Y):
    
    try:
        # splitting into test and train subsets
        x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=.1, 
                                                            random_state=42)

        params = {

#             # for GPU support
#             'tree_method': 'gpu_hist',

            ## "conservative" algorithms are ones that more tightly match the expected 
            # results. They have less fuzziness, so they return fewer results. 
            ## L2 regularization term on weights. Increasing this value will make model more
            # conservative.
            'lambda': trial.suggest_float('lambda', 1e-4, 10),

            ## L1 regularization term on weights. Increasing this value will make model more 
            # conservative.
            'alpha': trial.suggest_float('alpha', 1e-4, 10),

            ## colsample_bytree is the subsample ratio of columns when constructing each tree. 
            # Subsampling occurs once for every tree constructed.
            'colsample_bytree': trial.suggest_categorical('colsample_bytree', 
                                                          np.arange(.1, 1.1, 0.1)),

            # Subsample ratio of the training instances across all the training models.
            'subsample': trial.suggest_categorical('subsample', np.arange(.1, 1.1, 0.1)),

            'learning_rate': trial.suggest_categorical('learning_rate', [0.00001, .0003, .008, .002, 
                                                                         .01, 1, 10, 20  ]),
            'n_estimators': 3000, # number of trees in RF
            'max_depth': trial.suggest_categorical('max_depth', range(3, 13)),
            'random_state': trial.suggest_categorical('random_state', [10, 20, 30, 42, 100,
                                                                      1999]),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 200)
        }

        # building a model
        mod = xg.XGBRFRegressor(**params)
        
        # by the end of each trial, that particualr config will be evaluated on the 
        # `eval_test`.
        mod.fit(x_train, y_train, eval_set=[(x_test, y_test)], verbose=True)
        y_pred = mod.predict(x_test)
        rmse = mean_squared_error(y_test, y_pred)

        return rmse

    except Exception as e:
        print(e)
        
# Best Params (after trials):
# {'lambda': 7.98725011679705,
#  'alpha': 0.040527141040718306,
#  'colsample_bytree': 0.7000000000000001,
#  'subsample': 0.6,
#  'learning_rate': 1,
#  'max_depth': 6,
#  'random_state': 30,
#  'min_child_weight': 2}

- ### colsample_bytree: 
By limiting the number of features for building each tree we may end up with trees that gained different insights from the data. They learn how to optimise for the target variable using different set of features.

- ### learning_rate:
The learning rate is the shrinkage you do at every step you are making. If you make 1 step at eta = 1.00, the step weight is 1.00. If you make 1 step at eta = 0.25, the step weight is 0.25.

- ### subsample
 **Subsample ratio of the training instances.** Say, setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees and this will prevent overfitting. Subsampling will occur once in every boosting iteration.

In [11]:
find_params = optuna.create_study(direction='minimize')
find_params.optimize(objective, n_trials=10)

[I 2022-11-27 13:43:27,293] A new study created in memory with name: no-name-ca2f1763-f95d-46d1-a132-1cdd3c78b869


[0]	validation_0-rmse:0.17166


[I 2022-11-27 13:43:27,695] Trial 0 finished with value: 0.029468101700708505 and parameters: {'lambda': 0.9689480505243695, 'alpha': 3.173044444379095, 'colsample_bytree': 0.5, 'subsample': 1.0, 'learning_rate': 1, 'max_depth': 9, 'random_state': 10, 'min_child_weight': 182}. Best is trial 0 with value: 0.029468101700708505.


[0]	validation_0-rmse:0.15407


[I 2022-11-27 13:43:28,047] Trial 1 finished with value: 0.02373687413924265 and parameters: {'lambda': 9.994622389261266, 'alpha': 6.960846748880492, 'colsample_bytree': 0.2, 'subsample': 0.8, 'learning_rate': 1, 'max_depth': 8, 'random_state': 100, 'min_child_weight': 124}. Best is trial 1 with value: 0.02373687413924265.


...

[I 2022-11-27 13:43:30,348] Trial 7 finished with value: 15.652277131672841 and parameters: {'lambda': 3.2003198938720785, 'alpha': 6.155154635279718, 'colsample_bytree': 0.6, 'subsample': 0.8, 'learning_rate': 20, 'max_depth': 6, 'random_state': 1999, 'min_child_weight': 187}. Best is trial 1 with value: 0.02373687413924265.


[0]	validation_0-rmse:0.25037


[I 2022-11-27 13:43:30,816] Trial 8 finished with value: 0.0626874997652683 and parameters: {'lambda': 4.837233256806631, 'alpha': 1.6935175453669191, 'colsample_bytree': 0.30000000000000004, 'subsample': 0.4, 'learning_rate': 0.01, 'max_depth': 10, 'random_state': 10, 'min_child_weight': 3}. Best is trial 1 with value: 0.02373687413924265.


[0]	validation_0-rmse:0.25218


[I 2022-11-27 13:43:31,155] Trial 9 finished with value: 0.06359482118629305 and parameters: {'lambda': 3.688676157396771, 'alpha': 3.05869724539453, 'colsample_bytree': 0.2, 'subsample': 0.1, 'learning_rate': 0.008, 'max_depth': 11, 'random_state': 42, 'min_child_weight': 55}. Best is trial 1 with value: 0.02373687413924265.


In [12]:
best_params = find_params.best_trial.params
best_params

{'lambda': 9.994622389261266,
 'alpha': 6.960846748880492,
 'colsample_bytree': 0.2,
 'subsample': 0.8,
 'learning_rate': 1,
 'max_depth': 8,
 'random_state': 100,
 'min_child_weight': 124}

In [13]:
## Trials description

find_params.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.029468,2022-11-27 13:43:27.298136,2022-11-27 13:43:27.694664,0 days 00:00:00.396528,3.173044,0.5,0.968948,1.00000,9,182,10,1.0,COMPLETE
1,1,0.023737,2022-11-27 13:43:27.696665,2022-11-27 13:43:28.046698,0 days 00:00:00.350033,6.960847,0.2,9.994622,1.00000,8,124,100,0.8,COMPLETE
2,2,0.063595,2022-11-27 13:43:28.048692,2022-11-27 13:43:28.448722,0 days 00:00:00.400030,2.515173,0.2,6.876636,0.00030,10,96,100,0.3,COMPLETE
3,3,0.063595,2022-11-27 13:43:28.450722,2022-11-27 13:43:28.853753,0 days 00:00:00.403031,6.922615,0.5,8.816652,0.00001,8,23,100,0.5,COMPLETE
4,4,2.624164,2022-11-27 13:43:28.855753,2022-11-27 13:43:29.263797,0 days 00:00:00.408044,9.469821,0.6,8.690535,10.00000,6,73,42,0.8,COMPLETE
5,5,0.063423,2022-11-27 13:43:29.265784,2022-11-27 13:43:29.663814,0 days 00:00:00.398030,1.498772,0.6,7.997997,0.00200,4,40,30,0.3,COMPLETE
6,6,0.063595,2022-11-27 13:43:29.665814,2022-11-27 13:43:29.997840,0 days 00:00:00.332026,0.567165,0.1,2.747093,0.00001,8,131,100,0.2,COMPLETE
7,7,15.652277,2022-11-27 13:43:30.000840,2022-11-27 13:43:30.348867,0 days 00:00:00.348027,6.155155,0.6,3.200320,20.00000,6,187,1999,0.8,COMPLETE
8,8,0.062687,2022-11-27 13:43:30.350867,2022-11-27 13:43:30.815902,0 days 00:00:00.465035,1.693518,0.3,4.837233,0.01000,10,3,10,0.4,COMPLETE
9,9,0.063595,2022-11-27 13:43:30.818903,2022-11-27 13:43:31.154930,0 days 00:00:00.336027,3.058697,0.2,3.688676,0.00800,11,55,42,0.1,COMPLETE


In [14]:
## Optimization plot

optuna.visualization.plot_optimization_history(find_params)

In [15]:
## Optimization plot (params wise)

optuna.visualization.plot_slice(find_params)

In [16]:
# best params

best_params

{'lambda': 9.994622389261266,
 'alpha': 6.960846748880492,
 'colsample_bytree': 0.2,
 'subsample': 0.8,
 'learning_rate': 1,
 'max_depth': 8,
 'random_state': 100,
 'min_child_weight': 124}

In [17]:
## Contour plot

optuna.visualization.plot_contour(find_params, params=["alpha", "lambda"])

### # Building `XGBoost Regeressor` model with the best params computed via `Optuna` study:

In [18]:
## Splitting the data into train, test sets

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=42)

In [19]:
mod = xg.XGBRFRegressor(**best_params)
mod.fit(x_train, y_train)

XGBRFRegressor(alpha=6.960846748880492, base_score=0.5, booster='gbtree',
               callbacks=None, colsample_bylevel=1, colsample_bytree=0.2,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               lambda=9.994622389261266, learning_rate=1, max_bin=256,
               max_cat_to_onehot=4, max_delta_step=0, max_depth=8, max_leaves=0,
               min_child_weight=124, missing=nan, monotone_constraints='()',
               n_estimators=100, n_jobs=0, num_parallel_tree=100,
               objective='reg:squarederror', predictor='auto', random_state=100, ...)

In [20]:
## Training accuracy

mod.score(x_train, y_train)

-0.009829441068160083

In [21]:
## Test set accuracy

mod.score(x_test, y_test)

0.006285032591966555

#### NOTE: Optuna supports all libraries out there and that's why it's li'l more popular.

### # XGBoost Classifier using best params computed by the `Optuna` study:

In [22]:
ls

 Volume in drive D is Local Disk 
 Volume Serial Number is 407D-3EE5

 Directory of D:\data-science\machine-learning\in10__XGBoost

27-11-2022  13:43    <DIR>          .
27-11-2022  13:43    <DIR>          ..
30-10-2022  20:41    <DIR>          .ipynb_checkpoints
27-11-2022  13:43            16,821 in1__XGBoost, Optuna (hyperparameter tuning).ipynb
28-12-2019  17:56           100,951 winequality_red.csv
               2 File(s)        117,772 bytes
               3 Dir(s)  72,521,998,336 bytes free


In [23]:
## Loading the wine-qualoty dataset

df = pd.read_csv("winequality_red.csv")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [24]:
## description

df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [25]:
### Check for null values

df.isna().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

No null vals whatsoever!

In [26]:
## Feature Atrributes

x = df.drop("quality", axis=1)

X = np.array(x)
X.shape

(1599, 11)

In [27]:
## Label attribute

y = df["quality"]
y

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64

### # Gotta do `Encoding` for Label attribute:

In [28]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y)

LabelEncoder()

In [29]:
## Encoded Label attribute

Y_encoded = encoder.transform(y)

Y = Y_encoded.reshape(-1, 1)
Y.shape

(1599, 1)

In [38]:
### Train-Test split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=42)

In [40]:
### Hyperparameter tuning using `Optuna`
## Function for getting best params via trials

def objective_classification(trial):
    params = {
        'verbosity': 2,
        'objective': 'binary:logistics',
        'booster': trial.suggest_categorical('booster', ['dart', 'gbtree', 'gblinear']),
        'lambda': trial.suggest_float('lambda', 1e-4, 1),
        'alpha': trial.suggest_float('alpha', 1e-4, 1),
        'subsample': trial.suggest_float('subsample', .1, .5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', .1, .5)
    }
    
    ### There will be different params for a specific booster too!
    ## Then, let's not try to confuse the optuna.
    if params['booster'] in ['gbtree', 'dart']:
        params['gamma'] : trial.suggest_float('gamma', 1e-3, 4) # pruning hyperparam
        params['eta'] : trial.suggest_float('eta', .001, 5) # learning rate
        
    mod1 = xg.XGBRFClassifier(**params)
    mod1.fit(x_train, y_train, eval_set=[(x_test, y_test)], verbose=True)
    pred = mod1.predict(x_test)
    accuracy = mod1.score(x_test, y_test)
        
    return accuracy

- ### DART booster
XGBoost mostly combines a huge number of regression trees with a small learning rate. In this situation, trees added early are significant and trees added late are unimportant.

In [41]:
## Creating a study for trials

find_params = optuna.create_study(direction='minimize')
find_params.optimize(objective_classification, n_trials=100)

[I 2022-11-27 13:49:07,844] A new study created in memory with name: no-name-8c9dabcb-b852-4ce5-b150-b3d3bb05d06d


[13:49:07] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bynode", "colsample_bytree", "num_parallel_tree", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mlogloss:1.32202


[I 2022-11-27 13:49:07,893] Trial 0 finished with value: 0.40625 and parameters: {'booster': 'gblinear', 'lambda': 0.40900045877849694, 'alpha': 0.518314612788102, 'subsample': 0.18892181154049276, 'colsample_bytree': 0.15606199654588016}. Best is trial 0 with value: 0.40625.


[13:49:07] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[13:49:07] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 8 extra nodes, 0 pruned nodes, max_depth=4
[13:49:07] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 0 extra nodes, 0 pruned nodes, max_depth=0
[13:49:07] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 4 extra nodes, 0 pruned nodes, max_depth=2
[13:49:07] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 6 extra nodes, 0 pruned nodes, max_depth=3
[13:49:07] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 4 extra nodes, 0 pruned

[13:49:08] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 46 extra nodes, 0 pruned nodes, max_depth=6
[13:49:08] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 0 pruned nodes, max_depth=6
[13:49:08] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 22 extra nodes, 0 pruned nodes, max_depth=6
[13:49:08] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[13:49:08] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 38 extra nodes, 0 pruned nodes, max_depth=6
[13:49:08] INFO: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/tree/updater_prune.cc:101: tree pruning end, 48 extra nodes, 0 

...

[I 2022-11-27 13:49:20,216] Trial 97 finished with value: 0.40625 and parameters: {'booster': 'gblinear', 'lambda': 0.44894159873752304, 'alpha': 0.2763544764554027, 'subsample': 0.21122878407770218, 'colsample_bytree': 0.14758678649899573}. Best is trial 53 with value: 0.403125.


[13:49:20] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bynode", "colsample_bytree", "num_parallel_tree", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mlogloss:1.32122


[I 2022-11-27 13:49:20,266] Trial 98 finished with value: 0.40625 and parameters: {'booster': 'gblinear', 'lambda': 0.5144437886724545, 'alpha': 0.5850332316763676, 'subsample': 0.23201992703806318, 'colsample_bytree': 0.18141738625313344}. Best is trial 53 with value: 0.403125.


[13:49:20] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bynode", "colsample_bytree", "num_parallel_tree", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mlogloss:1.32061


[I 2022-11-27 13:49:20,312] Trial 99 finished with value: 0.40625 and parameters: {'booster': 'gblinear', 'lambda': 0.3776542622778435, 'alpha': 0.6469103833705923, 'subsample': 0.2468004261755195, 'colsample_bytree': 0.15384209161092569}. Best is trial 53 with value: 0.403125.


In [42]:
# best params

best_params = find_params.best_trial.params
best_params

{'booster': 'gblinear',
 'lambda': 0.3961013743274808,
 'alpha': 0.3766587986723646,
 'subsample': 0.3033520972713812,
 'colsample_bytree': 0.4432341786251721}

### for `10 trials`:

In [33]:
## best_params 

"""
{'booster': 'gblinear',
 'lambda': 0.8253859153587969,
 'alpha': 0.03963805273217429,
 'subsample': 0.2826168971239176,
 'colsample_bytree': 0.34604849552914263}
"""

"\n{'booster': 'gblinear',\n 'lambda': 0.8253859153587969,\n 'alpha': 0.03963805273217429,\n 'subsample': 0.2826168971239176,\n 'colsample_bytree': 0.34604849552914263}\n"

### for `100 trials`:

In [34]:
## best_params 

best_params

{'booster': 'gblinear',
 'lambda': 0.8938305430171075,
 'alpha': 0.09294540491635075,
 'subsample': 0.1369704115854055,
 'colsample_bytree': 0.14079493503086146}

### # Building `XGBoost Classfier` model with the `best_params`:

In [35]:
mod1 = xg.XGBRFClassifier(**best_params)
mod1.fit(x_train, y_train)

[13:43:46] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_bynode", "colsample_bytree", "num_parallel_tree", "subsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRFClassifier(alpha=0.09294540491635075, base_score=0.5, booster='gblinear',
                callbacks=None, colsample_bylevel=None,
                colsample_bytree=0.14079493503086146,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=None, gpu_id=-1, grow_policy=None,
                importance_type=None, interaction_constraints=None,
                lambda=0.8938305430171075, max_bin=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=None, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=0, num_parallel_tree=None,
                objective='multi:softprob', predictor=None, random_state=0,
                reg_alpha=0, ...)

In [36]:
## training accuracy:

mod1.score(x_train, y_train)

0.43080531665363564

In [37]:
## test accuracy:

mod1.score(x_test, y_test)

0.40625